
AI&DA18 Lab Exam
Name: Atharva M. Kulkarni
GR No: 11810384

Q7: Read the Dataset file “train_Titanic.csv” from the dataset folder. Design and train SVM machine learning
model on given columns(Survived-Dependent Column). Perform following task in jupyter

a. Fill the missing values

b. Do Column encoding (If necessary)

c. Split data in train test split (70:30 proportion)

d. Create 3 models on Linear Kernel, Polynomial Kernel and RBF Kernel

e. Find out the train score, test score for all models and compare the results..


In [85]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection, svm
import re
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
df = pd.read_csv('/content/drive/My Drive/train.csv')

In [87]:
print(df.head(10))

   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S
5            6         0       3  ...   8.4583   NaN         Q
6            7         0       1  ...  51.8625   E46         S
7            8         0       3  ...  21.0750   NaN         S
8            9         1       3  ...  11.1333   NaN         S
9           10         1       2  ...  30.0708   NaN         C

[10 rows x 12 columns]



Analyzing all the different Features / Labels and displaying a count of Nan values in the dataset


In [88]:
print(df.columns)
print(df.isnull().sum())

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64



The Cabin number could define where on the Ship was the person living and this in-turn could indicate whether the passenger had access to help immediately or whether he/she succumbed on impact.

So, here we categorize this column


In [89]:
Cabin_set = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

df['Cabin'] = df['Cabin'].fillna("U0")
df['Cabin_set'] = df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
df['Cabin_set'] = df['Cabin_set'].map(Cabin_set)
df['Cabin_set'] = df['Cabin_set'].fillna(0)

print(df.head())
print(df.columns)
print(df.isnull().sum())

   PassengerId  Survived  Pclass  ... Cabin Embarked  Cabin_set
0            1         0       3  ...    U0        S        8.0
1            2         1       1  ...   C85        C        3.0
2            3         1       3  ...    U0        S        8.0
3            4         1       1  ...  C123        S        3.0
4            5         0       3  ...    U0        S        8.0

[5 rows x 13 columns]
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Cabin_set'],
      dtype='object')
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         2
Cabin_set        0
dtype: int64



Replacing Nan 'Age' values with the median age


In [90]:
df['Age'].fillna(df['Age'].median(), inplace=True)
print(df.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
Cabin_set      0
dtype: int64



Dropping the redundant or less significant columns


In [91]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], 1)
print(df.columns)

df['Fare'] = df['Fare'].astype(int)
df['Cabin_set'] = df['Cabin_set'].astype(int)
df['Age'] = df['Age'].astype(int)
display(df)

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin_set'],
      dtype='object')


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_set
0,0,3,male,22,1,0,7,8
1,1,1,female,38,1,0,71,3
2,1,3,female,26,0,0,7,8
3,1,1,female,35,1,0,53,3
4,0,3,male,35,0,0,8,8
...,...,...,...,...,...,...,...,...
886,0,2,male,27,0,0,13,8
887,1,1,female,19,0,0,30,2
888,0,3,female,28,1,2,23,8
889,1,1,male,26,0,0,30,3


In [92]:
sex_numerical = {"male": 0, "female": 1}
df['Sex'] = df['Sex'].map(sex_numerical)
display(df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_set
0,0,3,0,22,1,0,7,8
1,1,1,1,38,1,0,71,3
2,1,3,1,26,0,0,7,8
3,1,1,1,35,1,0,53,3
4,0,3,0,35,0,0,8,8
...,...,...,...,...,...,...,...,...
886,0,2,0,27,0,0,13,8
887,1,1,1,19,0,0,30,2
888,0,3,1,28,1,2,23,8
889,1,1,0,26,0,0,30,3


In [93]:
X = df.drop(['Survived'], 1)
y = df['Survived']

In [94]:
X = np.array(X)
y = np.array(y)

In [95]:
X_train, X_test, y_train, y_test =  model_selection.train_test_split(X, y, test_size = 0.3, shuffle = True)


Here, we implemented SVM using the Linear, Polynomial and RBF kernels.

Results are summarized below


In [96]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
print("Train Accuracy: ", clf.score(X_train, y_train))
print("Test Accuracy: ", clf.score(X_test, y_test))

Train Accuracy:  0.8073836276083467
Test Accuracy:  0.7686567164179104


In [97]:
clf = svm.SVC(kernel='poly')
clf.fit(X_train, y_train)
print("Train Accuracy: ", clf.score(X_train, y_train))
print("Test Accuracy: ", clf.score(X_test, y_test))

Train Accuracy:  0.6629213483146067
Test Accuracy:  0.6268656716417911


In [99]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
print("Train Accuracy: ", clf.score(X_train, y_train))
print("Test Accuracy: ", clf.score(X_test, y_test))

Train Accuracy: 0.8756487811831469 0.6837881219903692
Test Accuracy: 0.8591123654886895 0.6529850746268657
